##### Imports

In [ ]:
from pathlib import Path

# RDKit as conformer generator
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign, Draw
from IPython.display import display, Markdown

# OPI imports for performing ORCA calculations and reading output
from opi.core import Calculator
from opi.input import simple_keywords
from opi.input.structures.structure import Structure
from opi.input.simple_keywords import BasisSet, Dft, ForceField, Scf, Task, Docker, SolvationModel, Solvent 
from opi.input.simple_keywords.goat import Goat
from opi.input.blocks import BlockDocker , BlockGeom, TSMode, Hybrid, geom_wrapper, BlockIrc 
from opi.input import blocks
from opi.input.blocks.block_geom import Scan
from opi.input.blocks.base import Block, InputFilePath
# for plotting results and visualization of molecules
from opi.input.blocks import BlockGeom, Hybrid
from opi.input.blocks.base import NumList
import py3Dmol
import pandas as pd
import matplotlib.pyplot as plt
from comp_chem_utils.utils import show_structure, setup_calc



## BnOH + H2O2

In [4]:
# Define SMILES for host and guest
host = "c1ccccc1CO"  # Benzyl alcohol
guest = "OO"         # Hydrogen peroxide

# Define base working directory
base_dir = Path("BnOH_plus_H2O2")
xtb_dir = base_dir / "xtb"

### Docking reactants

In [5]:
# Set up docking calculation
calc = setup_docking_calc(
    working_dir=xtb_dir,
    host_smiles=host,
    guest_smiles=guest,
    basename="BnOH_H2O2",
    visualize=False,
)

In [6]:
# run the docking calculation
calc.run()

In [3]:
show_structure("/home/madsr2d2/chem/comp_chem/BnOH_plus_H2O2/xtb/docking/BnOH_H2O2.docker.xyz", title="BnOH + H2O2 Docking Result")

### BnOH + H2O2 Docking Result

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### TS search

#### TS guess

In [119]:
# generate TS guess
ts_dir = xtb_dir / "ts"
ts_guess_dir  = ts_dir / "guess"

In [134]:
geom_block = BlockGeom()
geom_block.constraints = BlockGeom.constraint_str(["a 6 13 16 170.0 C", "B 13 16 1.7 C" , "a 13 16 17 170.0 c"])
geom_path = "/home/madsr2d2/chem/comp_chem/BnOH_plus_H2O2/xtb/docking/BnOH_H2O2.docker.xyz"
geom_block.recalc_hess = 100
geom_block.maxiter = 1000
sk_list = ["xtb", "alpb(water)", "opt", "TightSCF", "Slowconv"]

In [135]:
calc = setup_calc(
    working_dir=ts_guess_dir,
    structure=geom_path,
    basename="BnOH_H2O2_TS_guess",
    sk_list=sk_list,
    bk_list=geom_block,
)

### BnOH_H2O2_TS_guess

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [136]:
# run calculation
calc.run()

In [137]:

show_structure("/home/madsr2d2/chem/comp_chem/BnOH_plus_H2O2/xtb/ts/guess/BnOH_H2O2_TS_guess.xyz", title="TS guess")

### TS guess

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

#### TS Scan

In [8]:
# Scan dir
ts_dir = xtb_dir / "ts"
ts_scan_dir  = ts_dir / "scan_ts"

In [ ]:

geom_block = BlockGeom()
scan = BlockGeom.scan_str("B 13 16 = 1.7, 0.96, 20")
sk_list = ["xtb", "alpb(water)", "scanTS" , "slowconv", "TightSCF"]
geom_block.scan = scan
# geom_block.fullscan = True
geom_block.maxiter = 1000
# geom_block.recalc_hess = 100
# # geom_block.calc_hess = True
geom_block.cartfallback = True

geom_path = "/home/madsr2d2/chem/comp_chem/BnOH_plus_H2O2/xtb/ts/guess/BnOH_H2O2_TS_guess.xyz"

In [10]:

calc = setup_calc(
    working_dir=ts_scan_dir,
    structure=geom_path,
    basename="BnOH_H2O2_ts_scan",
    sk_list=sk_list,
    bk_list=geom_block,
)

### BnOH_H2O2_ts_scan

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
# run calculation
calc.run()

In [12]:
# Get the TS geometry
output = calc.get_output()
if output.terminated_normally():
    print("Calculation completed successfully.")
    for file in ts_scan_dir.glob("*.refined.xyz"):
      ts_geom = str(file).split('.')[0] + '.' + str(file).split('.')[1] + ".xyz"
      show_structure(ts_geom, title="TS scan result")
else:
    print("Calculation did not terminate normally.")
    

Calculation completed successfully.


### TS scan result

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### TS opt

In [13]:
# ts_opt dir
ts_dir = xtb_dir / "ts"
ts_opt_dir  = ts_dir / "ts_opt"

In [ ]:
geom_block = BlockGeom()
irc = BlockIrc(maxiter=100)
sk_list = ["xtb", "alpb(water)", "optTS" , "slowconv", "TightSCF", "freq", "IRC"]
geom_block.maxiter = 1000
geom_block.recalc_hess = 25
geom_block.calc_hess = True
geom_block.cartfallback = True
geom_path = ts_geom

In [15]:
calc = setup_calc(
    working_dir=ts_opt_dir,
    structure=geom_path,
    basename="BnOH_H2O2_ts_opt",
    sk_list=sk_list,
    bk_list=geom_block,
)

### BnOH_H2O2_ts_opt

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [16]:
# run calculation
calc.run()

In [17]:
# Get the TS geometry
output = calc.get_output()
if output.terminated_normally():
    print("Calculation completed successfully.")
    ts_geom = ts_opt_dir / "BnOH_H2O2_ts_opt.xyz"
    show_structure(ts_geom, title="TS_opt result")
    
else:
    print("Calculation did not terminate normally.")

Calculation completed successfully.


### TS_opt result

3Dmol.js failed to load for some reason. Please check your browser console for error messages.